In [1]:
import torch
import torchvision.transforms as tvt
from torch.utils.data import ConcatDataset, DataLoader
from torchmetrics import AUROC  # additional dependency
from torchvision.datasets import CIFAR10

from oodtk import ODIN
from oodtk.dataset.img import Textures, CIFAR10C, CIFAR10P, LSUNCrop, LSUNResize, TinyImageNetResize, TinyImageNetCrop
from oodtk.model import WideResNet
from oodtk.utils import is_unknown
from oodtk.transforms import ToRGB

In [2]:
torch.manual_seed(123)

mean = [x / 255 for x in [125.3, 123.0, 113.9]]
std = [x / 255 for x in [63.0, 62.1, 66.7]]

trans = tvt.Compose([ToRGB(), tvt.Resize((32,32)), tvt.ToTensor(), tvt.Normalize(mean, std)])

# setup data
dataset_train = CIFAR10(root="data", train=True, download=True, transform=trans)
dataset_in_test = CIFAR10(root="data", train=False, transform=trans)
dataset_out_test1 = Textures(root="data", download=True, transform=trans)
dataset_out_test2 = LSUNCrop(root="data", download=True, transform=trans)
dataset_out_test3 = LSUNResize(root="data", download=True, transform=trans)
dataset_out_test4 = TinyImageNetResize(root="data", download=True, transform=trans)
dataset_out_test5 = TinyImageNetCrop(root="data", download=True, transform=trans)
dataset_test = dataset_in_test + dataset_out_test1 + dataset_out_test2 + dataset_out_test3 + dataset_out_test4 + dataset_out_test5
train_loader = DataLoader(dataset_train, batch_size=128, num_workers=20)
test_loader = DataLoader(dataset_test, batch_size=64, num_workers=20)

Files already downloaded and verified


/home/ki/.local/share/anaconda3/envs/myenv2/lib/python3.8/site-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 16, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [3]:
model = WideResNet.from_pretrained("cifar10-pt", num_classes=10).cuda()

In [4]:
odin = ODIN(model, eps=0.002, norm_std=std)

In [5]:
from tqdm.notebook import tqdm

auroc = AUROC(num_classes=2)
model.eval()

with torch.no_grad():
    for eps in [0.0, 0.0004, 0.0008, 0.0014, 0.002, 0.0024, 0.0028, 0.0032, 0.0038, 0.0048]:
        odin.eps = eps
        for batch in tqdm(test_loader):
            x, y = batch
            x = x.cuda()
            y = y.cuda()
            auroc.update(odin.predict(x).detach(), is_unknown(y))

        print(eps, auroc.compute())
        auroc.reset()

/home/ki/.local/share/anaconda3/envs/myenv2/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


  0%|          | 0/870 [00:00<?, ?it/s]

/home/ki/.local/share/anaconda3/envs/myenv2/lib/python3.8/site-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 16, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


0.0 tensor(0.9217, device='cuda:0')


/home/ki/.local/share/anaconda3/envs/myenv2/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: `pos_label` automatically set 1.
  warnings.warn(*args, **kwargs)


...

0.0032 tensor(0.9320, device='cuda:0')


  0%|          | 0/870 [00:00<?, ?it/s]

0.0038 tensor(0.9257, device='cuda:0')


  0%|          | 0/870 [00:00<?, ?it/s]

0.0048 tensor(0.9130, device='cuda:0')


In [6]:
dataset_in_test = CIFAR10(root="data", train=False, transform=trans)
dataset_out_test = CIFAR10C(root="data", subset="all", download=True, transform=trans)
dataset_test = dataset_in_test +  dataset_out_test
test_loader = DataLoader(dataset_test, batch_size=128)

auroc = AUROC(num_classes=2)
model.eval()


for batch in tqdm(test_loader):
    x, y = batch
    x = x.cuda()
    y = y.cuda()

    auroc.update(odin(x), is_unknown(y))

print(eps, auroc.compute())


  0%|          | 0/79 [00:00<?, ?it/s]

TypeError: 'ODIN' object is not callable

In [ ]:
dataset_in_test = CIFAR10(root="data", train=False, transform=trans)
dataset_out_test = CIFAR10P(root="data", download=True, transform=trans)
dataset_test = dataset_in_test +  dataset_out_test
test_loader = DataLoader(dataset_test, batch_size=128)

model.eval()

for batch in tqdm(test_loader):
    x, y = batch
    x = x.cuda()
    y = y.cuda()

auroc.update(odin(x), is_unknown(y))

print(auroc.compute())

In [ ]:
dataset_out_test = CIFAR10C(root="data", subset="all", download=True, transform=trans)